In [ ]:
import QpixAsicArray as qparray
from QpixAsic import QPByte, ProcQueue
import random
import numpy as np
import pylab as p
import matplotlib.pyplot as plt
random.seed(2)

# Increase Jupyter notebook display 
from IPython.core.display import display, HTML, Latex
display(HTML("<style>.container { width:80% !important; }</style>")) #make the code cells bigger

In [ ]:
qpa = qparray.QpixAsicArray(3, 3, debug=0.0, deltaT=10e-6, timeEpsilon=1e-6)
qpa._daqNode._debugLevel = 0
qpa.Calibrate(1)
qpa.Calibrate(1)
# qpa.Calibrate(0.2)
print(qpa._daqNode.daqHits)

In [ ]:
# for asic in qpa:
#     asic.PrintStatus()
#     print()
#     print(f'does the asic have a connection to the North? {bool(asic.HasConnection(0))} \n')
#     key = f'({asic.row}, {asic.col})'
#     print(f'{key} has {asic.CountConnections()} connections \n')
#     print(f'the max local depth of the asic is: {asic._maxLocalDepth} \n')
#     print(f'the max queue depth of the asic is: {asic.maxConnDepths} \n')
    
    
    
# # qpa.readData()

In [ ]:
guessFrq = {}
for k, v in qpa._daqNode.pixelData.items():
    asic = k
    daqTicks = []
    asicTicks = []
    for daqTick, asicTick in v:
        daqTicks.append(daqTick)
        asicTicks.append(asicTick)
    print("asic ticks:", asicTicks)
    guessFrq[asic] = (asicTicks[-1] - asicTicks[-2]) / (daqTicks[1] - daqTicks[0])  * qpa._daqNode.fOsc
for asic in qpa:
    key = f"({asic.row},{asic.col})"
    print(f"{key} percent diff: {(guessFrq[key] - asic.fOsc) / asic.fOsc * 100:1.2E}")
#     print(f"testing {100:1.2E}")

In [ ]:
plt.figure(figsize=(6, 6))

for i in np.arange(10, 20, 10) * 1e-2:
    # print(f'the new deltaT is {i}')
    qpaNew = qparray.QpixAsicArray(3, 3, debug=0.0, deltaT=2e-6, timeEpsilon=1e-6)
    qpaNew.Calibrate(i)
    # qpaNew.Calibrate(i)
    
    guessFrq = {}
    percentDiffArray = []
    
    for k, v in qpaNew._daqNode.pixelData.items():
        asic = k
        # print(asic)
        daqTicks = []
        asicTicks = []
        for daqTick, asicTick in v:
            daqTicks.append(daqTick)
            asicTicks.append(asicTick)
            print("asic ticks:", asicTicks)
        guessFrq[asic] = (asicTicks[1] - asicTicks[0]) / (daqTicks[1] - daqTicks[0])  * qpaNew._daqNode.fOsc
        print(f'the guess frequency is {guessFrq[asic]} vs the actual frequency of {qpaNew.fNominal}')
        
#         percentdiff = np.absolute(guessFrq[asic] - asic.fOsc) / asic.fOsc * 100
    
    for asic in qpa:
        key = f"({asic.row},{asic.col})"
        percentdiff = (guessFrq[key] - asic.fOsc) / asic.fOsc * 100
        print(guessFrq[key])
        percentDiffArray.append(percentdiff)
        print(f"{key} percent diff: {percentdiff}")
        print(f"{key} percent diff: {(guessFrq[key] - asic.fOsc) / asic.fOsc * 100:1.2E}")
        
#         print(f'local transmissions are {asic._localTransmissions}')
    
    average = sum(percentDiffArray) / len(percentDiffArray) 
    
    plt.scatter(i * 1e6, average, marker='o', color='blue')
    plt.xlabel(r'$\mu$s')
    plt.ylabel('percent difference')
    plt.title('the percent difference')

In [ ]:
import QpixAsicArray as qparray
from QpixAsic import PixelHit, ProcQueue
import random
import numpy as np
import pylab as p
import matplotlib.pyplot as plt
random.seed(2)

# Increase Jupyter notebook display 
from IPython.core.display import display, HTML, Latex
display(HTML("<style>.container { width:80% !important; }</style>")) #make the code cells bigger


qpaNew = qparray.QpixAsicArray(3, 3, debug=0.0, deltaT=2e-6, timeEpsilon=1e-6)
qpaNew.Calibrate(0)

guessFrq = {}
percentDiffArray = []

for k, v in qpaNew._daqNode.pixelData.items():
    #pixelData stores the number of ticks relative to the asic
    asic = k
    print(f'the asic is {asic}')
    daqTicks = []
    asicTicks = []
    for daqTick, asicTick in v:
        daqTicks.append(daqTick)
        asicTicks.append(asicTick)
    print("asic ticks, first iteration:", asicTicks)


qpaNew.Calibrate(1.1)
for k, v in qpaNew._daqNode.pixelData.items():
    #pixelData stores the number of ticks relative to the asic 
    asic = k
    print(f'the asic is {asic}')
    daqTicks = []
    asicTicks = []
    for daqTick, asicTick in v:
        daqTicks.append(daqTick)
        asicTicks.append(asicTick)
    print("asic ticks, second iteration:", asicTicks)

    guessFrq[asic] = (asicTicks[1] - asicTicks[0]) / (daqTicks[1] - daqTicks[0])  * qpaNew._daqNode.fOsc
    # print(f'the guess frequency is {guessFrq[asic]} vs the actual frequency of {qpaNew.fNominal}')
    
#         percentdiff = np.absolute(guessFrq[asic] - asic.fOsc) / asic.fOsc * 100

for asic in qpaNew:
    key = f"({asic.row},{asic.col})"
    percentdiff = (guessFrq[key] - asic.fOsc) / asic.fOsc * 100
    print(guessFrq[key])
    percentDiffArray.append(percentdiff)
    print(f"{key} percent diff: {percentdiff}")    
#         print(f'local transmissions are {asic._localTransmissions}')

average = sum(percentDiffArray) / len(percentDiffArray) 

In [1]:
import QpixAsicArray as qparray
from QpixAsic import QPByte, ProcQueue

q = qparray.QpixAsicArray(3, 3, debug=0., deltaT=1e-7, timeEpsilon=1e-6)
q.Calibrate(.1)

# FirstDaqTick = [v[0][0] for k, v in q._daqNode.pixelData.items()]
# print(f'first daq ticks {FirstDaqTick}')
# FirstAsicTick = [v[0][1] for k, v in q._daqNode.pixelData.items()]
# print(f'first asic ticks {FirstAsicTick}')

Created ASIC at row 0 col 0 with frq: 50540252.16
Created ASIC at row 0 col 1 with frq: 51418163.32
Created ASIC at row 0 col 2 with frq: 48861019.67
Created ASIC at row 1 col 0 with frq: 53567846.97
Created ASIC at row 1 col 1 with frq: 48811674.28
Created ASIC at row 1 col 2 with frq: 53722404.56
Created ASIC at row 2 col 0 with frq: 49978222.29
Created ASIC at row 2 col 1 with frq: 49491134.36
Created ASIC at row 2 col 2 with frq: 51441719.96
performing calibration..
Line 148: 0


the daq interrogation arrived at asic (0, 0) at time 0
after array is processed there are 0 queue items
the in time for the interrog is 0 vs the current time 0
asic (0, 0) received an interrogation from (None, None)
After asic receives data, queue length is 2


the daq interrogation arrived at asic (0, 1) at time 5.223559217324677e-06
ASIC (None,None) STATE AsicState.Measure N_REMOTE(N,E,S,W) 0,0,0,0,t = 0 , trel =-3.9817767780737735e-09
ticks = 0
its new process items are [(, 1, <QpixAsic.QPByte object at

NameError: name 'exit' is not defined

In [20]:
print(q._daqNode.pixelData.items())
print(q._daqNode.daqHits)

for asic in q:
    for d in range(4):
        print(asic._remoteFifos[d]._full)
print(asic._localFifo._full)


dict_items([('(0,0)', [(550, 264)])])
335
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
